In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
from scipy import stats 
from scipy.io.arff import loadarff
from IPython.display import clear_output

#EEG (Preprocessing explained in another file)
eeg_raw_data, meta = loadarff('EEG Eye State.arff')
eeg_data = np.array(eeg_raw_data[meta.names()[0]].astype(float, copy = True)).reshape(14980,1)
for i in range(1,len(meta.names())-1):
    eeg_data = np.c_[eeg_data, np.array(eeg_raw_data[meta.names()[i]]).astype(float, copy = True)]
eye_state = np.array(eeg_raw_data[meta.names()[len(meta.names())-1]].astype(int, copy = True)).reshape(14980,1)

In [2]:
#Define PCA Function
def pca(data, num_of_prin_comp, data_orientation = "row"):
    num_of_data = len(data)
    dim_of_data = len(data[0])
    if data_orientation == "row":
        transposed_data = np.transpose(data) #Changes dataset so that data samples are column vectors
    mean = transposed_data.mean(1)  #Mean Vector
    centered_data = np.zeros((dim_of_data,num_of_data))

    for i in range(num_of_data):
        centered_data[:,i] = transposed_data[:,i] - mean  #Centering Data

    svd_u, svd_sigma, svd_v = np.linalg.svd(centered_data, full_matrices = True)  # Singular Value Decompostion

    u = np.zeros((dim_of_data,num_of_prin_comp))
    s = np.zeros((num_of_prin_comp,num_of_prin_comp))

    for i in range(dim_of_data):
        for j in range(num_of_prin_comp):
            u[i,j] = svd_u[i,j] #First r singular vectors of U
    for i in range(num_of_prin_comp):
        s[i,i] = svd_sigma[i] #Largest r singular values
    
    w = np.matrix(u)*np.matrix(s) #Principal Component Matrix with Principal Axes as Columns
    for i in range(num_of_prin_comp):
        w[:,i] = w[:,i]/np.linalg.norm(w[:,i]) #Normalizing Each Principal Component


    transformed_data = np.transpose(np.transpose(w)*centered_data) #Feature Vectors
    return transformed_data

Since the data is multidimensional, it could take much computational power to run the algorithm multiple times. That is why dimensionality reduction is used to simplify the data so that it represents 99% of the original data. This is done by the following equation:
<br>
$\frac{\sum_{i=1}^{k} \sigma^2_i}{\left \| \overline{X} \right \|^2_F}\geq .95$
<br>
This uses the singular value decomposition and the correlation of representation to the singular vlaues to find the top r principal components.

In [3]:
#Finding the smallest number of principal components for .99 Representation of Original Data:
m = len(eeg_data[0])
n = len(eeg_data)
centered = np.zeros((m,n))
for i in range(n):
    #Centering Training Data
    centered[:,i] = np.transpose(eeg_data)[:,i] - np.transpose(eeg_data).mean(1)
training_data_norm_squared = np.square(np.linalg.norm(centered))
svd_u, svd_sigma, svd_v = np.linalg.svd(centered, full_matrices = True)  # SVD

r = 0 #Top r principal components

for i in range(len(svd_sigma)):
    sum = 0
    representation = 0;
    for j in range(i+1):
        sum += np.square(svd_sigma[j])
    representation = sum/training_data_norm_squared
    if representation >= .99:
        r = i+1
        print("99% of the Original Data is represented by the top", r, "principal components")
        break

99% of the Original Data is represented by the top 3 principal components


Now using the top r principal components, use Principal Component Analysis to reduce the dimensions of the data.

In [4]:
transformed_eeg = pca(eeg_data, 3)

Defining the Gaussian Naive Bayes Classifier Function assumming Uniform Prior Distribution

In [5]:
#Gaussian Naive Bayes Classifier with Uniform Prior Distribution
import scipy.stats
def gaussian_naive_bayes(num_of_classes, training_data, training_labels, test_data):
    (n,m) = training_data.shape #n = number of training samples, m = dimensionality of data
    
    #Counting Training Data in Each Class
    class_size = np.zeros((num_of_classes)) #Defining number of training data in each class
    for i in range(n):
        class_size[int(training_labels[i])] += 1
    
    #Seperating Training Data
    seperated_training_data = np.zeros((num_of_classes), dtype = object)
    for i in range(num_of_classes):
        seperated_training_data[i] = np.zeros((int(class_size[i]), m))
    count = np.zeros((num_of_classes))
    for i in range(n):
        #Seperating the training data into arrays containing data for each class for easier calculation of class mean and variance
        seperated_training_data[int(training_labels[i])][int(count[int(training_labels[i])])] = training_data[i] 
        count[int(training_labels[i])] += 1
    
    #Mean and Variance Vectors
    mean = np.zeros((num_of_classes, m))
    variance = np.zeros((num_of_classes, m))
    for i in range(num_of_classes):
        mean[i] = seperated_training_data[i].mean(0) #Mean Vector for each class containing the mean of each feature in that class
        variance[i] = np.var(seperated_training_data[i], axis=0) #Variance Vector for each class
        
    #Defining Gaussian Distribution PDF (1-Dimensional)
    def pdf(x, me, var):
        return (1/np.sqrt(2*math.pi*var))*np.exp(-np.square(x-me)/(2*var))
        
    #Classifying Test Data
    classified_labels = np.ones((len(test_data)))
    for i in range(len(test_data)):
        probabilities = np.ones((num_of_classes)) #Contains the probability that test data belongs to each class
        for j in range(num_of_classes):
            for k in range(m):
                #Muptiplies all the probabilities of features conditioned on class
                #probabilities[j] *= pdf(test_data[i,k], mean[j][k], variance[j][k]) 
                probabilities[j] *= scipy.stats.norm(mean[j][k], variance[j][k]).pdf(test_data[i,k])
        classified_labels[i] = int(np.argmax(probabilities)) # Classifies the test data on the class with highest probability
        #print(probabilities, classified_labels[i], training_labels[i], classified_labels[i]==training_labels[i])
    
    return classified_labels

Defining a function that takes split data and labels to perform k-fold cross validation and return the mean and variance of the error on all the folds.

In [6]:
#Cross Vaidation Function
def cross_validation(k, split_data, split_labels, classifier_func = gaussian_naive_bayes):
    classification_rate = np.zeros((k)) #Array to hold classification rate of each fold
    
    for i in range(k):
        #Seperating split data into training and test sets (4 splits for training, 1 for testing)
        training = np.concatenate(np.delete(split_data, i))
        train_labels = np.concatenate(np.delete(split_labels, i))
        
        test = split_data[i]
        test_labels = split_labels[i]
        
        #Obtaining classified test labels
        if (classifier_func == "sklearn"):
            gnb = GaussianNB()
            gnb.fit(training, train_labels)
            classifier_labels = gnb.predict(test)
        else:
            classifier_labels = classifier_func(2, training, train_labels, test)
        
        #Calculating classification rate: (# of correctly classified test samples)/(total number of test samples)
        for j in range(len(classifier_labels)):
            if test_labels[j] == classifier_labels[j]:
                classification_rate[i] += 1
        classification_rate[i] /= len(classifier_labels)
        
    #Returning the mean and variance of the classification rates
    return np.mean(classification_rate), np.var(classification_rate)

Splitting the data into 10 folds for cross validation

In [7]:
k=10
n=len(eeg_data)

label_splits = np.zeros((k), dtype=object)
transformed_data_splits = np.zeros((k), dtype=object)

#Defining the splits
for i in range(0,k):
    label_splits[i] = np.array(eye_state[int((i*n)/k): int((i+1)*n/k)])
    transformed_data_splits[i] = np.array(transformed_eeg[int((i*n)/k): int((i+1)*n/k)])

In [8]:
mean, var = cross_validation(k, transformed_data_splits, label_splits)
print("Mean of classification rate is: ", mean)
print("Variance of classification rate is: ", var)

Mean of classification rate is:  0.4488651535380508
Variance of classification rate is:  0.0817910485008048
